In [1]:
import tweepy

%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from sklearn.model_selection import train_test_split


consumer_key = '9SRz5HMehrEVQf2m7AoN4shrq'
consumer_secret = 'zZa9j55quKFTmTwm4PKx4B6RUn3OyCsEtVJmvqbLAX9d8K3Adu'
access_token = '2801486303-55EJTjYXUPvw5uzXmRQV8wTHDmiLh70BJoASUj9'
access_token_secret = 'aRGfnR8N4if56loNt0yhwChXBe61go8qTpEanmXV2RBRp'

In [2]:
# Thank you, yanofsky! Adapted from: https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_names):

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    
    #initialize a list to hold all the tweepy Tweets
    df = pd.DataFrame()
    
    for screen_name in screen_names:
        
        alltweets = []

        #make initial request for most recent tweets (200 is the maximum allowed count)
        new_tweets = api.user_timeline(screen_name = screen_name ,count=200)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #save the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        #keep grabbing tweets until there are no tweets left to grab
        while len(new_tweets) > 0:
            print("getting tweets before %s" % (oldest))

            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

            #save most recent tweets
            alltweets.extend(new_tweets)

            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1

            print("...%s tweets downloaded so far" % (len(alltweets)))

        #transform the tweepy tweets into a 2D array that will populate the csv	
        outtweets = [[tweet.user.screen_name, tweet.text] for tweet in alltweets]

        df = df.append(pd.DataFrame(data=outtweets)).reset_index(drop=True)
        
    return df
    
    print('done!')

In [3]:
# Getting the most recent 3200 tweets from Donald Trump, Kanye West, Hillary Clinton, 
#  Taylor Swift
#tweets = get_all_tweets(['realDonaldTrump','justinbieber','hillaryclinton','katyperry'])

tweets = get_all_tweets(['acupofjoanne','figfolly'])

getting tweets before 517934632445439999
...80 tweets downloaded so far
getting tweets before 437402037047685120
...391 tweets downloaded so far
getting tweets before 271574014009409535
...568 tweets downloaded so far
getting tweets before 209725863636373503
...568 tweets downloaded so far


In [4]:
tweets[0].value_counts()

figfolly        568
aCupofJoanne     80
Name: 0, dtype: int64

In [5]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 2 columns):
0    648 non-null object
1    648 non-null object
dtypes: object(2)
memory usage: 10.2+ KB


In [6]:
tweets.columns = ['screenname','tweet']

In [7]:
tweets.head()

,screenname,tweet
0,aCupofJoanne,Thanks for sharing! 😸 https://t.co/QmvXtpTGRq
1,aCupofJoanne,"According to my research, sable cairn terriers..."
2,aCupofJoanne,Today is #NationalAdoptAShelterPetDay ! My lat...
3,aCupofJoanne,Thank you @thinkful for featuring my (literal)...
4,aCupofJoanne,RT @UCBerkeley: We stand with our undocumented...


In [8]:
tweets['tweet'] = tweets['tweet'].astype(str)


In [9]:
def text_cleaner(tweet):
    tweet = re.sub(r'http.*','',tweet)
    tweet = re.sub(r'bit/ly.*', "", tweet)
    tweet = re.sub(r'b\'', "", tweet)
    tweet = re.sub(r'b"', "", tweet)
    return tweet

In [10]:
for r in range(len(tweets['tweet'])):
    tweets['tweet'][r] = text_cleaner(tweets['tweet'][r])

#Dropping retweets    
tweets = tweets[tweets['tweet'].str.contains("RT") == False].reset_index(drop=True)
    

In [11]:
tweets.head(20)

,screenname,tweet
0,aCupofJoanne,Thanks for sharing! 😸
1,aCupofJoanne,"According to my research, sable cairn terriers..."
2,aCupofJoanne,Today is #NationalAdoptAShelterPetDay ! My lat...
3,aCupofJoanne,Thank you @thinkful for featuring my (literal)...
4,aCupofJoanne,@nicholashamlin Yaaas this is so unnecessarily...
5,aCupofJoanne,AHHH it's Gene Luen Yang! 😆#nationalbookfestival
6,aCupofJoanne,Came out publicly about my #shrimpchiplove ! ...
7,aCupofJoanne,"Asian. Bachelorette. Is. Everything. \n\nAlso,..."
8,aCupofJoanne,👏🏻👏🏻👏🏻 for defining p-values and the publicati...
9,aCupofJoanne,Excited for this and also for being two degree...


Helpful link about disabling piplines: https://spacy.io/usage/processing-pipelines#disabling

In [12]:
# Concatenizing all tweets into one text and parsing.
nlp = spacy.load('en', disable=['parser'])

text = tweets['tweet'].str.cat()
text = nlp(text)

In [13]:
# Parse tweets
nlp = spacy.load('en')

parsed = []

for r in tweets['tweet']:
    p = nlp(r)
    parsed.append(p)
    
    

In [14]:
tweets['parsed'] = parsed

In [15]:
tweets.head()

,screenname,tweet,parsed
0,aCupofJoanne,Thanks for sharing! 😸,"(Thanks, for, sharing, !, 😸)"
1,aCupofJoanne,"According to my research, sable cairn terriers...","(According, to, my, research, ,, sable, cairn,..."
2,aCupofJoanne,Today is #NationalAdoptAShelterPetDay ! My lat...,"(Today, is, #, NationalAdoptAShelterPetDay, !,..."
3,aCupofJoanne,Thank you @thinkful for featuring my (literal)...,"(Thank, you, @thinkful, for, featuring, my, (,..."
4,aCupofJoanne,@nicholashamlin Yaaas this is so unnecessarily...,"(@nicholashamlin, Yaaas, this, is, so, unneces..."


# Bag of Words
Let's use bag of words! For each tweet, we will count how many times each word appears and use those counts as features.  
Maybe we can distinguish between the politicians (Trump, Clinton and Barack) and the celebrities (Kanye, Taylor Swift, and Ellen Degeneres.)

In [16]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(tweets, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = tweets['parsed']
    df['text_source'] = tweets['screenname']
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [17]:
# Finding the top 1000 common words in all the tweets.

common_words = bag_of_words(text)

In [18]:
common_words

['-PRON-',
 'be',
 ' ',
 '\n',
 'not',
 "'s",
 'the',
 'get',
 'good',
 'go',
 'coffee',
 '\n\n',
 'like',
 'time',
 'twitter',
 'know',
 'food',
 'have',
 'thank',
 'today',
 'and',
 'right',
 'do',
 'what',
 'watch',
 'hour',
 '@jp_delahoussaye',
 'school',
 'feel',
 'paper',
 '@scars33',
 '@nmbou',
 'look',
 'new',
 'bagel',
 'tweet',
 'email',
 'thing',
 'need',
 'grad',
 '@vyvngo',
 'find',
 'day',
 'take',
 'a',
 'life',
 'come',
 'say',
 'to',
 '$',
 'pretty',
 'read',
 'health',
 'love',
 'wear',
 'word',
 'want',
 'make',
 'student',
 'work',
 '@nprnew',
 'there',
 'will',
 '5',
 'book',
 'wait',
 'picture',
 'send',
 'think',
 'eat',
 'friend',
 'amp',
 'public',
 'one',
 'great',
 'apha14',
 'probably',
 'this',
 'no',
 'hair',
 'study',
 'realize',
 'old',
 'year',
 'happy',
 'haha',
 'exam',
 'sleep',
 '@divyatuli_5',
 'yes',
 '@kwutnow',
 'follow',
 'dog',
 'big',
 'social',
 '@wastenotoc',
 'that',
 'sure',
 'buy',
 'people',
 'use',
 'drink',
 '3',
 'write',
 'excited',

In [19]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(tweets, common_words)
word_counts.head()

Processing row 0
Processing row 500


,-PRON-,be,,,not,'s,the,get,good,go,...,competition,sunday,hb,cent,grandma,gee,knittaplz,wheremystichesat@scars33,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Thanks, for, sharing, !, 😸)",aCupofJoanne
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(According, to, my, research, ,, sable, cairn,...",aCupofJoanne
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Today, is, #, NationalAdoptAShelterPetDay, !,...",aCupofJoanne
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Thank, you, @thinkful, for, featuring, my, (,...",aCupofJoanne
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(@nicholashamlin, Yaaas, this, is, so, unneces...",aCupofJoanne


In [20]:
word_counts.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Columns: 1002 entries, -PRON- to text_source
dtypes: object(1002)
memory usage: 4.5+ MB


In [21]:
#word_counts.to_csv('tweet_word_counts.csv')


# Models!

In [22]:
# Train-test split
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0,
                                                    stratify=Y
                                                   )

In [23]:
y_train.value_counts()

figfolly        319
aCupofJoanne     37
Name: text_source, dtype: int64

In [24]:
y_test.value_counts()

figfolly        214
aCupofJoanne     24
Name: text_source, dtype: int64

### Random forest

In [25]:
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9971910112359551

Test set score: 0.9327731092436975


In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(rfc, X, Y, cv=5)

array([0.89166667, 0.89915966, 0.89915966, 0.89830508, 0.91525424])

In [27]:
y_pred = rfc.fit(X_train, y_train).predict(X_train)

In [28]:
Counter(y_pred)

Counter({'aCupofJoanne': 36, 'figfolly': 320})

### Multinomial Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(356, 1000) (356,)
Training set score: 0.9859550561797753

Test set score: 0.9117647058823529


In [30]:
cross_val_score(lr, X, Y, cv=5)

array([0.89166667, 0.90756303, 0.90756303, 0.90677966, 0.90677966])

### Gradient Boost

In [31]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))


Training set score: 1.0

Test set score: 0.9327731092436975


In [32]:
cross_val_score(clf, X, Y, cv=5)

array([0.89166667, 0.93277311, 1.        , 0.94067797, 0.91525424])

### SVM

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# Code adapted from http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
# Set the parameters by cross-validation
def gridsearch(X_train, y_train, X_test, y_test):
    
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9],
                     'C': [1, 10, 100, 1000, 10000]}]

    scores = ['precision', 'recall']

    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                           scoring='%s_macro' % score)
        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
        print()

In [34]:
# To speed up production, we are going to take the X and Y test dataset, split it for testing and training
#  and use grid search on that.  A smaller dataset will hopefully make this run faster.
X2_train, X2_test, y2_train, y2_test = train_test_split(X_test, y_test, test_size=0.4,
                                                    random_state=0, stratify=y_test)

In [35]:
y2_train.value_counts(1)

figfolly        0.901408
aCupofJoanne    0.098592
Name: text_source, dtype: float64

In [74]:
y2_train

17     aCupofJoanne
283        figfolly
565        figfolly
581        figfolly
480        figfolly
396        figfolly
260        figfolly
347        figfolly
93         figfolly
382        figfolly
395        figfolly
512        figfolly
370        figfolly
295        figfolly
104        figfolly
358        figfolly
348        figfolly
108        figfolly
417        figfolly
530        figfolly
143        figfolly
0      aCupofJoanne
254        figfolly
335        figfolly
497        figfolly
564        figfolly
578        figfolly
134        figfolly
352        figfolly
276        figfolly
           ...     
101        figfolly
239        figfolly
36     aCupofJoanne
273        figfolly
45     aCupofJoanne
503        figfolly
481        figfolly
119        figfolly
111        figfolly
550        figfolly
501        figfolly
47     aCupofJoanne
326        figfolly
388        figfolly
46     aCupofJoanne
85         figfolly
171        figfolly
551        figfolly
549        figfolly


In [36]:
#Show warning once
import warnings
warnings.filterwarnings('once')

gridsearch(X2_train, y2_train, X2_test, y2_test)

# Tuning hyper-parameters for precision



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classificatio

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classificatio

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classificatio

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classificatio

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classificatio

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.451 (+/-0.012) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 1, 'gamma': 1e-06, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 1, 'gamma': 1e-07, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 1, 'gamma': 1e-08, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 1, 'gamma': 1e-09, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 1e-06, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 1e-07, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 1e-08, 'kernel': 'rbf'}
0.451 (+/-0.012) for {'C': 10, 'gamma': 1e-09, 

In [72]:
y_pred = svc.fit(X2_train, y2_train).predict(X2_train)

In [73]:
Counter(y_pred)

Counter({'aCupofJoanne': 14, 'figfolly': 128})

In [38]:
svc = SVC(C=1000, gamma=0.001, kernel='rbf')
svc.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [41]:
cross_val_score(svc, X, Y, cv=5)

array([0.89166667, 0.94117647, 0.94957983, 0.91525424, 0.91525424])

# tf-idf

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [75]:
X_train, X_test = train_test_split(tweets['tweet'], test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(#max_df=0.2, # drop words that occur in more than half the paragraphs
                             #min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
tweets_tfidf=vectorizer.fit_transform(tweets)
print("Number of features: %d" % tweets_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(tweets_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_train[2])
print('Tf_idf vector:', tfidf_bypara[0])

Number of features: 3
Original sentence: Today is #NationalAdoptAShelterPetDay ! My latest @thinkful project looks at pet adoption data from @austinanimals… 
Tf_idf vector: {'screenname': 1.0}


# Word2vec

In [66]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
twts = []
for tweet in tweets['parsed']:
    tweet = [
        token.lemma_.lower()
        for token in tweet
        if not token.is_stop
        and not token.is_punct
    ]
    twts.append(tweet)


print(twts[20])
print('We have {} tweets and {} tokens.'.format(len(twts), len(text)))

['@zoho', 'new', 'user', 'contactmanager', 'have', 'trouble', 'uploading', 'contact', 'any', 'tip']
We have 594 tweets and 9538 tokens.


In [69]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    twts,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [70]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(vocab)

dict_keys(['thank', ' ', 'read', 'today', '-pron-', 'look', 'time', 'be', 'come', '\n\n', '@jp_delahoussaye', 'grad', 'school', 'like', 'tweet', 'twitter', 'the', 'say', 'to', '\n', 'health', 'and', 'in', 'not', 'food', 'email', 'get', 'a', 'new', 'have', "'s", 'love', 'good', '@vyvngo', 'apha14', 'go', 'coffee', 'just', '$', 'what', 'right', 'know', 'oh', '@kwutnow', 'wear', 'thing', 'pretty', 'picture', 'word', 'want', 'do', 'need', 'make', 'feel', 'find', 'day', '@scars33', 'life', 'watch', 'year', 'take', 'hour', 'student', 'send', 'work', 'paper', '@nmbou', '@jabantao', 'bagel'])


In [71]:
print(model.wv.most_similar(positive=['year', 'time'], negative=['hour']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('tweet', 'twitter'))
print(model.wv.similarity('paper', 'student'))

# One of these things is not like the other...
print(model.doesnt_match("school student paper bagel".split()))

[('-pron-', 0.23321452736854553), ('bagel', 0.2298261523246765), (' ', 0.2166469693183899), ('be', 0.21484529972076416), ('pretty', 0.19631724059581757), ('the', 0.18546323478221893), ('right', 0.1717025339603424), ('\n\n', 0.1667495220899582), ('not', 0.1619897186756134), ('have', 0.16156628727912903)]
0.006546927591919074
-0.02478316157365136
student


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()


Oof. Not a good model with this tiny dataset.